<a href="https://colab.research.google.com/github/kuostar0620-jpg/114-1KUO-REPO-/blob/main/41371124hweek2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [ ]:
import pandas as pd
# read data and put it in a dataframe
# 在 google 工作表載入 gsheets
gsheets = gc.open_by_url('https://docs.google.com/spreadsheets/d/1eTIiO-nrJWfa9Oi61BqryQRSKQAOg73MHKv2dK5_yjw/edit?usp=sharing')


In [ ]:
gsheets

<Spreadsheet 'log (2)' id:1eTIiO-nrJWfa9Oi61BqryQRSKQAOg73MHKv2dK5_yjw>

In [ ]:
# 從 gsheets 的 All-whiteboard-device 載入 sheets
sheets = gsheets.worksheet('log (2)').get_all_values()
# 將 sheets1 資料載入 pd 的 DataFrame 進行分析
df = pd.DataFrame(sheets[1:], columns=sheets[0])
# 取得最前面的5筆資料
df.head()

,日期,時間,品項,金額,餐別
0,2025-09-11,10:56:45,蘋果,33,早餐
1,2025-09-11,11:18:58,apple,34,早餐
2,2025-09-11,11:26:46,2,100,早餐
3,2025-09-11,11:28:21,apple,200,早餐


In [1]:
import gradio as gr
import pandas as pd
import os
from datetime import datetime
import pytz

# --- 檔案路徑設定 ---
LOG_FILE = "expense_log.txt"

# --- 核心功能: 判斷餐別 ---
def get_meal_type(time_str):
    """
    根據時間字串判斷餐別。
    """
    try:
        time_obj = datetime.strptime(time_str, '%H:%M:%S').time()

        if datetime.strptime('00:00:00', '%H:%M:%S').time() <= time_obj <= datetime.strptime('11:59:59', '%H:%M:%S').time():
            return "早餐"
        elif datetime.strptime('12:00:00', '%H:%M:%S').time() <= time_obj <= datetime.strptime('15:59:59', '%H:%M:%S').time():
            return "午餐"
        elif datetime.strptime('16:00:00', '%H:%M:%S').time() <= time_obj <= datetime.strptime('23:59:59', '%H:%M:%S').time():
            return "晚餐"
        else:
            return "其他"

    except (ValueError, IndexError):
        return "未知"

# --- 功能: 讀取記錄並生成 DataFrame ---
def read_log_to_dataframe():
    """
    讀取記錄檔案，並將資料轉換成一個 Pandas DataFrame。
    """
    if not os.path.exists(LOG_FILE):
        return pd.DataFrame()

    data_list = []
    try:
        with open(LOG_FILE, "r", encoding="utf-8") as file:
            for line in file:
                line = line.strip()
                if not line:
                    continue
                parts = line.split(", ")
                entry = {}
                for part in parts:
                    key_value = part.split(": ", 1)
                    if len(key_value) == 2:
                        key = key_value[0]
                        value = key_value[1]
                        entry[key] = value
                if entry:
                    data_list.append(entry)

        if not data_list:
            return pd.DataFrame()

        df = pd.DataFrame(data_list)
        if '金額' in df.columns:
            df['金額'] = pd.to_numeric(df['金額'], errors='coerce')
        return df

    except Exception:
        return pd.DataFrame()

# --- 介面功能: 記錄新的支出 ---
def record_new_entry(item, amount, date_str, time_str):
    """
    讓使用者輸入並記錄一筆新的支出，並更新 DataFrame。
    """
    if not item or amount is None:
        return read_log_to_dataframe(), "錯誤：品項和金額為必填項目。"

    try:
        amount = float(amount)
        if amount <= 0:
            return read_log_to_dataframe(), "錯誤：金額必須是正數。"

        meal_type = get_meal_type(time_str)
        log_entry = f"日期: {date_str}, 時間: {time_str}, 品項: {item}, 金額: {amount:.2f}, 餐別: {meal_type}\n"

        with open(LOG_FILE, "a", encoding="utf-8") as file:
            file.write(log_entry)

        return read_log_to_dataframe(), f"記錄成功！\n品項：{item}\n金額：{amount:.2f}\n餐別：{meal_type}"

    except ValueError:
        return read_log_to_dataframe(), "錯誤：金額輸入無效，請輸入數字。"
    except Exception as e:
        return read_log_to_dataframe(), f"發生錯誤：{e}"

# --- 介面功能: 顯示所有記錄和統計 ---
def view_records():
    """
    顯示所有記錄的表格，並進行簡單的統計。
    """
    df = read_log_to_dataframe()
    if df.empty:
        return df, "目前沒有任何記錄。"

    total_amount = df['金額'].sum()
    stats_markdown = f"**總支出：** {total_amount:.2f} 元\n\n"

    if '餐別' in df.columns:
        meal_summary = df.groupby('餐別')['金額'].sum().reset_index()
        meal_summary.columns = ['餐別', '總金額']
        stats_markdown += "**各餐別的總支出：**\n"
        stats_markdown += meal_summary.to_markdown(index=False)

    return df, stats_markdown

# --- 介面功能: 刪除單筆記錄 ---
def delete_entry(index):
    """
    根據索引刪除一筆記錄。
    """
    df = read_log_to_dataframe()
    if df.empty:
        return df, "錯誤：目前沒有任何記錄可以刪除。"

    try:
        index_to_delete = int(index)
        if 0 <= index_to_delete < len(df):
            df = df.drop(df.index[index_to_delete])
            df.to_csv(LOG_FILE, index=False, header=False, sep=",", encoding="utf-8-sig")

            # 重新寫入檔案
            with open(LOG_FILE, "w", encoding="utf-8") as file:
                for _, row in df.iterrows():
                    log_entry = f"日期: {row['日期']}, 時間: {row['時間']}, 品項: {row['品項']}, 金額: {row['金額']:.2f}, 餐別: {row['餐別']}\n"
                    file.write(log_entry)

            return read_log_to_dataframe(), f"已成功刪除第 {index_to_delete} 筆記錄。"
        else:
            return df, "錯誤：無效的記錄編號，請輸入 0 到 {len(df)-1} 之間的數字。"
    except ValueError:
        return df, "錯誤：輸入無效，請輸入一個數字。"
    except Exception as e:
        return df, f"發生錯誤：{e}"

# --- 介面功能: 匯出 CSV ---
def export_csv():
    """
    將記錄檔案轉換為 CSV 格式。
    """
    df = read_log_to_dataframe()
    if df.empty:
        return None

    csv_file = "expense_log.csv"
    df.to_csv(csv_file, index=False, encoding="utf-8-sig")
    return csv_file

# --- 建立 Gradio 介面 ---

taipei_tz = pytz.timezone('Asia/Taipei')
now_taipei = datetime.now(taipei_tz)
default_date = now_taipei.strftime('%Y-%m-%d')
default_time = now_taipei.strftime('%H:%M:%S')

with gr.Blocks(theme=gr.themes.Soft(), title="支出追蹤器") as demo:
    gr.Markdown("# 簡潔支出追蹤器")
    gr.Markdown("這個應用程式能幫助您輕鬆記錄每日支出。")

    with gr.Tabs():
        with gr.TabItem("記錄新支出"):
            gr.Markdown("### 記錄一筆新的支出")
            with gr.Row():
                item_input = gr.Textbox(label="品項", placeholder="例如：咖啡")
                amount_input = gr.Number(label="金額", placeholder="例如：120")
            with gr.Row():
                date_input = gr.Textbox(label="日期", value=default_date)
                time_input = gr.Textbox(label="時間", value=default_time)

            record_button = gr.Button("記錄支出")
            record_status = gr.Textbox(label="狀態")

        with gr.TabItem("查看與管理記錄"):
            gr.Markdown("### 所有記錄")
            df_output = gr.DataFrame(headers=["日期", "時間", "品項", "金額", "餐別"], label="支出記錄", interactive=False)
            stats_output = gr.Markdown("統計數據將在此顯示。")
            refresh_button = gr.Button("更新記錄")

            gr.Markdown("### 刪除記錄")
            gr.Markdown("您可以通過輸入表格最左側的索引號（從 0 開始）來刪除記錄。")
            delete_index = gr.Number(label="要刪除的記錄編號", precision=0, value=None)
            delete_button = gr.Button("刪除記錄")
            delete_status = gr.Textbox(label="刪除狀態")

        with gr.TabItem("匯出資料"):
            gr.Markdown("### 匯出為 CSV 檔案")
            gr.Markdown("將所有記錄匯出為 CSV 檔案，方便您在其他軟體中分析。")
            csv_button = gr.Button("匯出 CSV")
            csv_file = gr.File(label="下載您的 CSV 檔案")

    # 連結功能與介面
    record_button.click(
        fn=record_new_entry,
        inputs=[item_input, amount_input, date_input, time_input],
        outputs=[df_output, record_status]
    )

    refresh_button.click(
        fn=view_records,
        outputs=[df_output, stats_output]
    )

    delete_button.click(
        fn=delete_entry,
        inputs=[delete_index],
        outputs=[df_output, delete_status]
    )

    csv_button.click(
        fn=export_csv,
        inputs=None,
        outputs=[csv_file]
    )

    demo.load(fn=view_records, outputs=[df_output, stats_output])

if __name__ == "__main__":
    demo.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://b665dfd5c12debfc59.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
